In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


## Create *data* folder

In [2]:
# excel_file = 'ilc_scp27_spreadsheet.xlsx'
# all_sheets = pd.read_excel(excel_file, sheet_name=None)
# sheets = all_sheets.keys()

# for sheet_name in sheets:
#     sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
#     sheet.to_csv("data/%s.csv" % sheet_name, index=False)

## Finding useful information about each sheet

In [3]:
xslx = pd.read_excel('ilc_scp27_spreadsheet.xlsx')
xslx = xslx.drop(xslx.index[0:13])
xslx = xslx.reset_index(drop=True)

#make the first row the column names
xslx.columns = xslx.iloc[0]
xslx = xslx.drop(xslx.index[0])
xslx = xslx.reset_index(drop=True) 
xslx

,NaN,Contents,Time frequency,International Standard Classification of Education (ISCED 2011),Sex,Age class,Unit of measure,Time
0,NaN,Sheet 1,Annual,All ISCED 2011 levels,Total,From 16 to 24 years,Percentage,2022
1,NaN,Sheet 2,Annual,All ISCED 2011 levels,Total,From 16 to 29 years,Percentage,2022
2,NaN,Sheet 3,Annual,All ISCED 2011 levels,Total,From 16 to 44 years,Percentage,2022
3,NaN,Sheet 4,Annual,All ISCED 2011 levels,Total,From 16 to 64 years,Percentage,2022
4,NaN,Sheet 5,Annual,All ISCED 2011 levels,Total,16 years or over,Percentage,2022
...,...,...,...,...,...,...,...,...
199,NaN,Sheet 200,Annual,Tertiary education (levels 5-8),Females,From 55 to 64 years,Percentage,2022
200,NaN,Sheet 201,Annual,Tertiary education (levels 5-8),Females,From 65 to 74 years,Percentage,2022
201,NaN,Sheet 202,Annual,Tertiary education (levels 5-8),Females,65 years or over,Percentage,2022
202,NaN,Sheet 203,Annual,Tertiary education (levels 5-8),Females,From 75 to 84 years,Percentage,2022


In [4]:
one_hot_encoded_xslx = pd.get_dummies(xslx)
sum_values = one_hot_encoded_xslx.sum()

for index in sum_values.index:
    value = sum_values.loc[index]
    if value == xslx.shape[0]:
        print(f"Column: {index} has all values the same")

Column: Time frequency_Annual has all values the same
Column: Unit of measure_Percentage has all values the same
Column: Time_2022 has all values the same


In [5]:
#drop the columns that are NaN or not needed
xslx_without_nan = xslx.drop(xslx.columns[0], axis=1)
xslx_summary = xslx_without_nan.drop(columns=['Time frequency', 'Unit of measure', 'Time'], axis=1)
xslx_summary.rename(columns={'International Standard Classification of Education (ISCED 2011)': 'ISCED_2011'}, inplace=True)
xslx_summary.rename(columns={'Age class': 'Age'}, inplace=True)
xslx_summary.columns = xslx_summary.columns.str.lower()
xslx_summary

,contents,isced_2011,sex,age
0,Sheet 1,All ISCED 2011 levels,Total,From 16 to 24 years
1,Sheet 2,All ISCED 2011 levels,Total,From 16 to 29 years
2,Sheet 3,All ISCED 2011 levels,Total,From 16 to 44 years
3,Sheet 4,All ISCED 2011 levels,Total,From 16 to 64 years
4,Sheet 5,All ISCED 2011 levels,Total,16 years or over
...,...,...,...,...
199,Sheet 200,Tertiary education (levels 5-8),Females,From 55 to 64 years
200,Sheet 201,Tertiary education (levels 5-8),Females,From 65 to 74 years
201,Sheet 202,Tertiary education (levels 5-8),Females,65 years or over
202,Sheet 203,Tertiary education (levels 5-8),Females,From 75 to 84 years


## Show unique categories in selected columns

In [6]:
print(xslx_summary['isced_2011'].unique())
print(xslx_summary['sex'].unique())
print(xslx_summary['age'].unique())

['All ISCED 2011 levels'
 'Less than primary, primary and lower secondary education (levels 0-2)'
 'Upper secondary and post-secondary non-tertiary education (levels 3 and 4)'
 'Tertiary education (levels 5-8)']
['Total' 'Males' 'Females']
['From 16 to 24 years' 'From 16 to 29 years' 'From 16 to 44 years'
 'From 16 to 64 years' '16 years or over' 'From 20 to 64 years'
 'From 25 to 29 years' 'From 25 to 34 years' 'From 30 to 54 years'
 'From 35 to 44 years' 'From 45 to 54 years' 'From 45 to 64 years'
 'From 55 to 64 years' 'From 65 to 74 years' '65 years or over'
 'From 75 to 84 years' '75 years or over']


## Substract useful data from each sheet

In this case for each sheet I am substracting three categories: *sex*, *age class* and *ISCED 2011 (education level)*. I decided to compare Poland and European Union average values in four ranges (read books in last 12 months):
 - none
 - less than five
 - more than five, less than 9
 - ten or more

In [7]:
def generate_description(df):
    first_value = str(df.at[0, 'Unnamed: 2'])
    second_value = str(df.at[1, 'Unnamed: 2'])
    third_value = str(df.at[2, 'Unnamed: 2']) 
    descripted_features = (first_value, second_value, third_value)
    return descripted_features

In [8]:
dictionary = {}
def features_dictionary(sheet_name, description):
    dictionary.update({sheet_name: description})
    return dictionary

In [9]:
import os
import pandas as pd

# Paths to the data folders
data_folder = 'data'
data_renamed_folder = 'data_new'

# Create the data_renamed folder if it doesn't exist
os.makedirs(data_renamed_folder, exist_ok=True)

# List all files in the data folder
files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

In [10]:
counter = 1

for file in files:
    file_path = os.path.join(data_folder, file)
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue
    
    df = pd.read_csv(file_path)

    filtered_df = pd.concat([df[df.apply(lambda row: row.astype(str).str.contains('(ISCED 2011)').any(), axis=1)],
                            df[df.apply(lambda row: row.astype(str).str.contains('Sex').any(), axis=1)],
                            df[df.apply(lambda row: row.astype(str).str.contains('Age class').any(), axis=1)],
                            df[df.apply(lambda row: row.astype(str).str.contains('N_BOOK').any(), axis=1)],
                            df[df.apply(lambda row: row.astype(str).str.contains('European Union').any(), axis=1)],
                            df[df.apply(lambda row: row.astype(str).str.contains('Poland').any(), axis=1)]] 
                            )

    filtered_df = filtered_df.reset_index(drop=True)
    description = generate_description(filtered_df)
    features_dictionary(file, description)
    filtered_df.drop(filtered_df.index[:3], inplace=True)

    filtered_df.columns = filtered_df.iloc[0]
    filtered_df = filtered_df.drop(filtered_df.index[0])
    filtered_df = filtered_df.reset_index(drop=True)
    filtered_df.to_csv('data_new/' + file, index=False)
    counter += 1

In [11]:
education_levels = xslx_summary['isced_2011'].unique().tolist()
gender = xslx_summary['sex'].unique().tolist()
age_group = xslx_summary['age'].unique().tolist() 
print(education_levels)
print(gender)
print(age_group)

['All ISCED 2011 levels', 'Less than primary, primary and lower secondary education (levels 0-2)', 'Upper secondary and post-secondary non-tertiary education (levels 3 and 4)', 'Tertiary education (levels 5-8)']
['Total', 'Males', 'Females']
['From 16 to 24 years', 'From 16 to 29 years', 'From 16 to 44 years', 'From 16 to 64 years', '16 years or over', 'From 20 to 64 years', 'From 25 to 29 years', 'From 25 to 34 years', 'From 30 to 54 years', 'From 35 to 44 years', 'From 45 to 54 years', 'From 45 to 64 years', 'From 55 to 64 years', 'From 65 to 74 years', '65 years or over', 'From 75 to 84 years', '75 years or over']


In [12]:
def create_dataframe_to_plot(features_dict, selected_education_level=None, selected_gender=None, selected_age_group=None):
    # List all files in the data_renamed folder
    files = [f for f in os.listdir(data_renamed_folder) if f.endswith('.csv')]
    data_folder = 'data_new'
    
    # Loop through each item in the features dictionary
    for item in features_dict.items():
        
        item_name = item[0]
        item_description = item[1]
        item_description_education = item_description[0]
        item_description_gender = item_description[1]
        item_description_age = item_description[2]

        if selected_gender == item_description_gender and selected_education_level == item_description_education and selected_age_group == item_description_age:
            #print(f"Processing: {item_name} features with value {item_description_gender}")
            print(f"kot numer {item_name} widzi człowieka {item_description_gender}, który ma wykształcenie na poziomie {item_description_education} i jest w grupie wiekowej {item_description_age}")
            file_path = os.path.join(data_folder, item_name)
            print(file_path)
            df = pd.read_csv(file_path)

    return df
       

In [58]:
created1 = create_dataframe_to_plot(dictionary, selected_gender='Males', selected_age_group='From 16 to 64 years', selected_education_level='All ISCED 2011 levels')
created2 = create_dataframe_to_plot(dictionary, selected_gender='Females', selected_age_group='From 16 to 64 years', selected_education_level='All ISCED 2011 levels')

kot numer Sheet 21.csv widzi człowieka Males, który ma wykształcenie na poziomie All ISCED 2011 levels i jest w grupie wiekowej From 16 to 64 years
data_new\Sheet 21.csv
kot numer Sheet 38.csv widzi człowieka Females, który ma wykształcenie na poziomie All ISCED 2011 levels i jest w grupie wiekowej From 16 to 64 years
data_new\Sheet 38.csv


In [70]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filter datasets for European Union
eu_data1 = created1[created1['N_BOOK (Labels)'] == 'European Union - 27 countries (from 2020)']
eu_data2 = created2[created2['N_BOOK (Labels)'] == 'European Union - 27 countries (from 2020)']

# Filter datasets for Poland
pl_data1 = created1[created1['N_BOOK (Labels)'] == 'Poland']
pl_data2 = created2[created2['N_BOOK (Labels)'] == 'Poland']

# Combine the filtered datasets
eu_combined = pd.concat([eu_data1, eu_data2])
pl_combined = pd.concat([pl_data1, pl_data2])

eu_combined = eu_combined.drop(columns=['N_BOOK (Labels)'])
pl_combined = pl_combined.drop(columns=['N_BOOK (Labels)'])
eu_combined = eu_combined.reset_index(drop=True)
pl_combined = pl_combined.reset_index(drop=True)

# Create subplots: use 'domain' type for pie charts
fig_male = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                    subplot_titles=['European Union', 'Poland'])

# Add the European Union donut chart
fig_male.add_trace(go.Pie(labels=eu_combined.columns, values=eu_combined.values[0], name='EU'),
              row=1, col=1)


# Add the Poland donut chart
fig_male.add_trace(go.Pie(labels=pl_combined.columns, values=pl_combined.values[0], name='Poland'),
              row=1, col=2)


# Update layout for the figure
fig_male.update_layout(
    title_text='Males 16-64 age group',
    annotations=[dict(text='EU', x=0.25, y=1, font_size=20, showarrow=False),
                 dict(text='Poland', x=0.75, y=1, font_size=20, showarrow=False)]
)

# Show the figure
fig_male.show()

In [72]:
# Create subplots: use 'domain' type for pie charts
fig_female = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                    subplot_titles=['European Union', 'Poland'])

# Add the European Union donut chart
fig_female.add_trace(go.Pie(labels=eu_combined.columns, values=eu_combined.values[1], name='EU'),
              row=1, col=1)


# Add the Poland donut chart
fig_female.add_trace(go.Pie(labels=pl_combined.columns, values=pl_combined.values[1], name='Poland'),
              row=1, col=2)


# Update layout for the figure
fig_female.update_layout(
    title_text='Females 16-64 age group',
    annotations=[dict(text='EU', x=0.25, y=1, font_size=20, showarrow=False),
                 dict(text='Poland', x=0.75, y=1, font_size=20, showarrow=False)]
)

# Show the figure
fig_female.show()